---
# **4. Фильтрация портфеля**

Удаляются активы, которые:
1. Имеют недостаточно данных ($T < 100$)
2. Показывают экстремальную волатильность ($\sigma > 50\%$)
3. Нестационарны по GARCH ($\alpha + \beta \geq 0.995$)
4. Являются дубликатами других активов ($|\rho| > 0.98$)

При высокой корреляции между двумя активами удаляется актив с **большей волатильностью**
(он более рискован, менее полезен для диверсификации).

### Проверка размера

$$n_{\text{final}} \geq 3$$

Если портфель меньше → выбросить исключение.

### Выходные данные

- Очищенные доходности: $\mathbf{R}_{\text{filtered}}$
- Ковариационная матрица: $\Sigma_{\text{filtered}}$
- Веса волатильностей (GARCH или Sample)

---


In [ ]:
MIN_OBSERVATIONS = 100
MAX_VOLATILITY = 50.0
MAX_CORRELATION = 0.98
MIN_PORTFOLIO_SIZE = 3


print(f"  Параметры фильтрации:")
print(f"   MIN_OBS={MIN_OBSERVATIONS}, MAX_VOL={MAX_VOLATILITY}%, MAX_CORR={MAX_CORRELATION}\n")

volatilities_final = garch_results['volatilities_final']
garch_params = garch_results['params']
garch_usage = garch_results['garch_usage']
failed_tickers = garch_results['failed_tickers']

n_garch_success = len(garch_params)
print(f"   Успешно GARCH: {n_garch_success}")
print(f"   Sample fallback: {len(returns.columns) - n_garch_success}\n")

initial_assets = returns.columns.tolist()
assets_to_keep = []
removed = {}


for ticker in initial_assets:
    reasons = []

    n_obs = len(returns)

    if n_obs < MIN_OBSERVATIONS:
        reasons.append(f"n_obs={n_obs}")

    if ticker not in volatilities_final.index or pd.isna(volatilities_final[ticker]):
        reasons.append("vol=NaN")
    else:
        vol = volatilities_final[ticker]

        if len(garch_params) > 0 and ticker in garch_params.index:
            persistence = garch_params.loc[ticker, 'persistence']
            if persistence >= 0.995:
                reasons.append("non-stationary")

        if vol > MAX_VOLATILITY:
            reasons.append(f"σ={vol:.1f}%")

    if reasons:
        removed[ticker] = reasons
        print(f" {ticker}: {' | '.join(reasons)}")
    else:
        assets_to_keep.append(ticker)
        source = garch_usage.get(ticker, "unknown")
        vol = volatilities_final[ticker]
        print(f" {ticker}: σ={vol:>6.2f}% | n={n_obs:>3d} | [{source}]")

print(f"\n {len(assets_to_keep)}/{len(initial_assets)} активов прошли базовую фильтрацию\n")

if len(assets_to_keep) < 2:
    print(f"  Слишком мало активов после базовой фильтрации: {len(assets_to_keep)}")
    print(f"   Попробуйте ослабить критерии: MAX_VOL={MAX_VOLATILITY}, MAX_CORR={MAX_CORRELATION}, MIN_OBS={MIN_OBSERVATIONS}")

duplicates_removed = []

if len(assets_to_keep) >= 2:
    corr_matrix = returns[assets_to_keep].corr()

    high_corr_found = False

    for i in range(len(assets_to_keep)):
        for j in range(i + 1, len(assets_to_keep)):
            ticker_i = assets_to_keep[i]
            ticker_j = assets_to_keep[j]

            if ticker_i in duplicates_removed or ticker_j in duplicates_removed:
                continue

            corr = abs(corr_matrix.loc[ticker_i, ticker_j])

            if corr > MAX_CORRELATION:
                high_corr_found = True

                vol_i = volatilities_final[ticker_i]
                vol_j = volatilities_final[ticker_j]

                if vol_i > vol_j:
                    removed_ticker = ticker_i
                    kept_ticker = ticker_j
                else:
                    removed_ticker = ticker_j
                    kept_ticker = ticker_i

                duplicates_removed.append(removed_ticker)
                removed[removed_ticker] = [f"dup_{kept_ticker}_ρ={corr:.3f}"]

                print(f" {removed_ticker} ↔ {kept_ticker}: ρ={corr:.3f} (удален {removed_ticker}, σ={volatilities_final[removed_ticker]:.2f}%)")

    assets_to_keep = [t for t in assets_to_keep if t not in duplicates_removed]

    if duplicates_removed:
        print(f"\n Удалено дубликатов: {len(duplicates_removed)}\n")
    else:
        print(f" Дубликатов не найдено\n")
else:
    print(f"  Недостаточно активов для проверки корреляций (найдено {len(assets_to_keep)})\n")

if len(assets_to_keep) < MIN_PORTFOLIO_SIZE:
    error_msg = (
        f" Ошибка: Недостаточно активов для оптимизации!\n"
        f"   Требуется минимум: {MIN_PORTFOLIO_SIZE}\n"
        f"   Найдено: {len(assets_to_keep)}\n\n"
        f"   Ослабьте критерии фильтрации:\n"
        f"   - MAX_VOLATILITY: {MAX_VOLATILITY}% → увеличить\n"
        f"   - MAX_CORRELATION: {MAX_CORRELATION} → увеличить\n"
        f"   - MIN_OBSERVATIONS: {MIN_OBSERVATIONS} → уменьшить"
    )
    raise ValueError(error_msg)


print(f" Размер портфеля адекватен: {len(assets_to_keep)} активов (требуется {MIN_PORTFOLIO_SIZE})\n")

filtered_returns = returns[assets_to_keep].copy()
filtered_cov = cov_matrix.loc[assets_to_keep, assets_to_keep].copy()
filtered_mu = mu[assets_to_keep].copy()
filtered_annual_vol = annual_vol[assets_to_keep].copy()
filtered_garch_vols = volatilities_final[assets_to_keep].copy()
filtered_names = assets_to_keep

print(f" Размер данных:")
print(f"   Возвращаемость: {filtered_returns.shape}")
print(f"   Ковариация: {filtered_cov.shape}")
print(f"   Активов: {len(filtered_names)}\n")

eigenvalues = np.linalg.eigvalsh(filtered_cov.values)
min_eig = eigenvalues.min()
max_eig = eigenvalues.max()
condition_number = max_eig / min_eig if min_eig > 0 else np.inf


status = "✅" if min_eig > 1e-10 else "⚠️ "
print(f"{status} Ковариационная матрица:")
print(f"   λ_min = {min_eig:.2e}")
print(f"   λ_max = {max_eig:.2e}")
print(f"   Condition number = {condition_number:.2e}\n")

print(f" Активы в портфеле ({len(filtered_names)}):\n")

for ticker in filtered_garch_vols.sort_values(ascending=False).index:
    vol_garch = filtered_garch_vols[ticker]
    vol_sample = filtered_annual_vol[ticker]
    ret = filtered_mu[ticker]
    source = garch_usage.get(ticker, "unknown")

    emoji_ret = "🟢" if ret > 0 else "🔴"
    emoji_src = "📊" if source == "GARCH" else "📈"

    print(f"   {emoji_ret} {emoji_src} {ticker:6s}: σ={vol_garch:>6.2f}% (Sample: {vol_sample:>6.2f}%), μ={ret*100:>7.2f}% [{source}]")


print(f"\n Статистика волатильностей:")
print(f"   Min:    {filtered_garch_vols.min():>6.2f}%")
print(f"   Median: {filtered_garch_vols.median():>6.2f}%")
print(f"   Mean:   {filtered_garch_vols.mean():>6.2f}%")
print(f"   Max:    {filtered_garch_vols.max():>6.2f}%")


print(f"\n Статистика доходностей:")
print(f"   Min:    {filtered_mu.min()*100:>7.2f}%")
print(f"   Mean:   {filtered_mu.mean()*100:>7.2f}%")
print(f"   Max:    {filtered_mu.max()*100:>7.2f}%")

if removed:
    print(f"\n Удалено активов: {len(removed)}")
    print(f"   Причины удаления:")

    for ticker in sorted(removed.keys()):
        reasons = removed[ticker]
        reasons_str = " | ".join(reasons)
        print(f"   - {ticker}: {reasons_str}")

n_garch = sum(1 for x in filtered_garch_vols.index if garch_usage.get(x) == "GARCH")
n_sample = len(filtered_names) - n_garch

print(f"\n Источники волатильностей в итоговом портфеле:")
print(f"   GARCH: {n_garch}/{len(filtered_names)}")
print(f"   Sample (fallback): {n_sample}/{len(filtered_names)}")

print(f" Переменные:")
print(f"   filtered_returns     : {filtered_returns.shape} — лог-доходности")
print(f"   filtered_cov         : {filtered_cov.shape} — ковариационная матрица (годовая)")
print(f"   filtered_mu          : ({len(filtered_mu)},) — ожидаемые доходности")
print(f"   filtered_annual_vol  : ({len(filtered_annual_vol)},) — Sample волатильности")
print(f"   filtered_garch_vols  : ({len(filtered_garch_vols)},) — финальные волатильности (GARCH + Sample)")
print(f"   filtered_names       : {filtered_names}")
print(f"\n Итого:")
print(f"   Активов в портфеле: {len(filtered_names)}/{len(initial_assets)}")
print(f"   Активов удалено: {len(removed)}")